In [1]:
from keras.applications.resnet50 import ResNet50
# load model
model = ResNet50()
# summarize the model
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 10s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 6

In [3]:
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Model
import cv2
import os
import time
import numpy as np
from imutils import paths

folder structure:

/faces

    /mask
    
    /no_mask

In [28]:
# mask = 1
# no_mask = 0
images_path = "/home/tl/darknet/faces/"
data = []
labels = []
num_classes = 2
for img in paths.list_images(images_path):
    image = cv2.imread(img)
    image = cv2.resize(image,(32,32))
    image = img_to_array(image)
    data.append(image)
    label = img.split("/")[-2]
    label = 1 if label == "mask" else 0
    labels.append(label)
    
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
(X_train, X_test, y_train, y_test) = train_test_split(data,labels, test_size=0.25, random_state=42)

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [17]:
# image_input = Input(shape=(32, 32, 3))

# model = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
# model.summary()
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

### Transfer Learning

In [29]:
model = ResNet50(weights='imagenet',include_top=False)
model.summary()
last_layer = model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(last_layer)
# add fully-connected & dropout layers
x = Dense(512, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu',name='fc-2')(x)
x = Dropout(0.5)(x)
# a softmax layer for 4 classes
out = Dense(num_classes, activation='softmax',name='output_layer')(x)

# this is the model we will train
custom_resnet_model2 = Model(inputs=model.input, outputs=out)

custom_resnet_model2.summary()

for layer in custom_resnet_model2.layers[:-6]:
	layer.trainable = False

custom_resnet_model2.layers[-1].trainable

custom_resnet_model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
hist = custom_resnet_model2.fit(X_train, y_train, batch_size=16, epochs=12, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_resnet_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_14[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________________________________

Train on 4764 samples, validate on 1588 samples
Epoch 1/12
4764/4764 [==============================] - 10s 2ms/step - loss: 0.5914 - accuracy: 0.7853 - val_loss: 0.6586 - val_accuracy: 0.6662
Epoch 2/12
4764/4764 [==============================] - 8s 2ms/step - loss: 0.3600 - accuracy: 0.8541 - val_loss: 0.6614 - val_accuracy: 0.7009
Epoch 3/12
4764/4764 [==============================] - 7s 2ms/step - loss: 0.3525 - accuracy: 0.8541 - val_loss: 0.6211 - val_accuracy: 0.7009
Epoch 4/12
4764/4764 [==============================] - 7s 2ms/step - loss: 0.3362 - accuracy: 0.8663 - val_loss: 0.6413 - val_accuracy: 0.7009
Epoch 5/12
4764/4764 [==============================] - 7s 1ms/step - loss: 0.3023 - accuracy: 0.8816 - val_loss: 0.6502 - val_accuracy: 0.7009
Epoch 6/12
4764/4764 [==============================] - 6s 1ms/step - loss: 0.2889 - accuracy: 0.8837 - val_loss: 0.7562 - val_accuracy: 0.7009
Epoch 7/12
4764/4764 [==============================] - 7s 2ms/step - loss: 0.2831 - ac

In [50]:
# saving model
custom_resnet_model2.save("ResNet50_model.h5")
print("Saved model to disk")

Saved model to disk


In [6]:
from keras.models import load_model
    model = load_model('ResNet50_model.h5')

img_path = '/home/tl/darknet/covid_video/no_mask/44_1080_1080.jpg'
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
np.argmax(pred)

1

In [49]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
# from imagenet_utils import decode_predictions
img_path = '/home/tl/darknet/faces/no_mask/0Frame1344.jpg'
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = custom_resnet_model2.predict(x)
# model.predict_classes(x_test_reshaped)
print(type(preds))
print(preds)
print(max(preds[0]))

<class 'numpy.ndarray'>
[[0.65516967 0.34483033]]
0.65516967


In [47]:
n = np.array([0.9,0.2])
max(n)

0.9

image generator object which performs random rotations, shifts, flips, crops, and sheers on our image dataset.

In [ ]:
# construct the image generator for data augmentation
# not used in this training
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [23]:
# #Training the classifier alone
# image_input = Input(shape=(32, 32, 3))

# model = ResNet50(input_tensor=image_input, include_top=False,weights='imagenet')
# model.summary()
# # last_layer = model.get_layer('avg_pool').output
# last_layer = model.layers[-1]
# x= Flatten(name='flatten')(last_layer)
# out = Dense(num_classes, activation='softmax', name='output_layer')(x)
# custom_resnet_model = Model(inputs=image_input,outputs= out)
# custom_resnet_model.summary()

# for layer in custom_resnet_model.layers[:-1]:
#     layer.trainable = False

# custom_resnet_model.layers[-1].trainable

# custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# t=time.time()
# hist = custom_resnet_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))
# print('Training time: %s' % (t - time.time()))
# (loss, accuracy) = custom_resnet_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

# print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

/home/tl/lnt/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

ValueError: Layer flatten was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.core.Activation'>. Full input: [<keras.layers.core.Activation object at 0x7f526a256e48>]. All inputs to the layer should be tensors.

In [5]:
from keras.models import load_model
model = load_model('ResNet50_model.h5')

dir_path = "/home/tl/darknet/covid_video/no_mask/"

total = len(list(paths.list_images(dir_path)))
correct = 0
incorrect = 0
for img_path in paths.list_images(dir_path):
    img = image.load_img(img_path, target_size=(32, 32))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    pred = model.predict(x)
    label = np.argmax(pred) 
    if img_path.split("/")[-2]=="no_mask" and label == 0:
        correct += 1
    elif img_path.split("/")[-2]=="mask" and label == 1:
        correct += 1
    else:
        incorrect += 1
        
print("total",total)
print("correct",correct)
print("incorrect",incorrect)
print("correctly classified:"+str(correct/total))
print("not correctly classified:"+str(incorrect/total))

total 89
correct 2
incorrect 87
correctly classified:0.02247191011235955
not correctly classified:0.9775280898876404
